In [48]:
import random
import json
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
session_id = random.randint(0,1000000)

TaskObject = '''    {
        "TaskInstanceId": 20,
        "TaskMasterId": -2,
        "TaskStatus": "InProgress",
        "TaskType": "Azure Storage to SQL Database",
        "Enabled": 1,
        "ExecutionUid": "b829721c-f297-49eb-8436-c33e27005971",
        "NumberOfRetries": 0,
        "DegreeOfCopyParallelism": 1,
        "KeyVaultBaseUrl": "https://mst-stg-kv-ads-pnu0.vault.azure.net/",
        "ScheduleMasterId": "4",
        "TaskGroupConcurrency": "10",
        "TaskGroupPriority": 0,
        "TaskExecutionType": "ADF",
        "DataFactory": {
            "Id": 1,
            "Name": "mst-stg-adf-ads-pnu0",
            "ResourceGroup": "adsgftera2",
            "SubscriptionId": "035a1364-f00d-48e2-b582-4fe125905ee3",
            "ADFPipeline": "GPL_AzureBlobFS_Parquet_AzureSqlTable_NA_Azure",
            "TaskDatafactoryIR": "Azure"
        },
        "Source": {
            "System": {
                "SystemId": 4,
                "SystemServer": "https://mststgdlsadspnu0adsl.dfs.core.windows.net",
                "AuthenticationType": "MSI",
                "Type": "ADLS",
                "Username": null,
                "Container": "datalakeraw"
            },
            "Instance": {
                "SourceRelativePath": "samples/"
            },
            "DataFileName": "SalesLT.Customer.chunk_1.parquet",
            "DeleteAfterCompletion": "false",
            "MaxConcurrentConnections": 0,
            "Recursively": "false",
            "RelativePath": "samples/",
            "Type": "Parquet"
        },
        "Target": {
            "System": {
                "SystemId": 4,
                "SystemServer": "https://mststgdlsadspnu0adsl.dfs.core.windows.net",
                "AuthenticationType": "MSI",
                "Type": "ADLS",
                "Username": null,
                "Container": "datalakeraw"
            },
            "Instance": {
                "SourceRelativePath": "samples/"
            },
            "DataFileName": "SalesLT-Customer-Delta",
            "DeleteAfterCompletion": "false",
            "MaxConcurrentConnections": 0,
            "Recursively": "false",
            "RelativePath": "deltalake/samples/",
            "Type": "Delta"
        }
    }'''


StatementMeta(mststgsynspads, 6, 50, Finished, Available)

In [49]:

TaskObjectJson = json.loads(TaskObject)
Source = TaskObjectJson['Source']['System']['Container'] + "@" + TaskObjectJson['Source']['System']['SystemServer'].replace("https://","") + "/"
Target = TaskObjectJson['Target']['System']['Container'] + "@" + TaskObjectJson['Target']['System']['SystemServer'].replace("https://","") + "/"

Source = Source + TaskObjectJson['Source']['RelativePath'] + "/" + TaskObjectJson['Source']['DataFileName']
Target = Target + TaskObjectJson['Target']['RelativePath'] + "/" + TaskObjectJson['Target']['DataFileName']

#remove any double slashes
Source.replace('//', '')
TargetContainer.replace('//', '')

#add abfss
Source = "abfss://" + Source
Target = "abfss://" + Target

print (Source)
print (Target)


StatementMeta(mststgsynspads, 6, 51, Finished, Available)

abfss://datalakeraw@mststgdlsadspnu0adsl.dfs.core.windows.net/samples//SalesLT.Customer.chunk_1.parquet
abfss://datalakeraw@mststgdlsadspnu0adsl.dfs.core.windows.net/deltalake/samples//SalesLT-Customer-Delta

In [66]:
from delta.tables import *
df = spark.read.load(Source, format='parquet')
sql = 'describe detail "' + Target + '"'
if (spark.sql(sql).collect()[0].asDict()['format'] == 'delta'):
    print("Table already exists. Performing Merge")
    olddt = DeltaTable.forPath(spark, Target)
    (olddt
    .alias("oldData") 
    .merge(df.alias("newData"), "oldData.CustomerID = newData.CustomerID")
    .whenMatchedUpdate(set = {"FirstName": col("newData.FirstName")})
    .whenNotMatchedInsert(values = {"CustomerID": col("newData.CustomerID"), "FirstName":
                                    col("newData.FirstName")})
    .execute()
    )
else:
    print("Table does not exist.")    
    df.write.format("delta").save(Target)

StatementMeta(mststgsynspads, 6, 68, Finished, Available)

Table already exists. Performing Merge

In [67]:

olddt.history().show(20, 1000, False)

#spark.sql("CREATE TABLE SalesLTCustomer USING DELTA LOCATION '{0}'".format(TargetFile))

StatementMeta(mststgsynspads, 6, 69, Finished, Available)

+-------+-------------------+------+--------+---------+------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|          timestamp|userId|userName|operation|                                         operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|                                                                                                                                                                                                  operationMetrics|
+-------+-------------------+------+--------+---------+------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------

In [ ]:
# Execute Upsert
(old_deltaTable
 .alias("oldData") 
 .merge(newIncrementalData.alias("newData"), "oldData.id = newData.id")
 .whenMatchedUpdate(set = {"name": col("newData.name")})
 .whenNotMatchedInsert(values = {"id": col("newData.id"), "name":
                                col("newData.name")})
 .execute()
)

# Display the records to check if the records are Merged
display(spark.read.format("delta").load("/data/events_old/"))